# Plantdoc-dataset

In [26]:
!cp /kaggle/input/leafclip-model/model.py /kaggle/working/
!cp /kaggle/input/utils-py/ultis.py /kaggle/working/
!cp /kaggle/input/class-prediction-json/class_description.json /kaggle/working/
!cp /kaggle/input/leafclip-cations-features-pth/leafclip_captions_features.pth /kaggle/working/

In [27]:
# # Import and load
# from model import ImageEncoder
# from ultis import predict_image_v2
# import torch

# # Load pretrained weights
# model = ImageEncoder()
# model.load_state_dict(torch.load(
#     "/kaggle/input/image-encoder-leafclip/image_encoder_leaf_clip.pth",
#     map_location="cpu"
# ))
# model.eval()

# # Inference 
# img_path = "/kaggle/input/plantdoc-dataset/train/Apple_Scab_Leaf/train_Apple Scab Leaf_10.jpg"
# preds = predict_image_v2(model, img_path, "class_description.json", top_k=5)

# print("Top 5 predictions:")
# for text, prob in preds:
#     print(f"{text}: {prob*100:.2f}%")


In [28]:
import os

root = '/kaggle/input/plantdoc-dataset/train'
rows = []

for dirpath, _, files in os.walk(root):
    class_name = os.path.basename(dirpath)
    for fname in files:
        # Check if the filename contains '_2.' before the extension
        if '_2.' in fname and fname.lower().endswith(('.jpg', '.png')):
            img_path = os.path.join(dirpath, fname)
            rows.append((class_name, img_path))
            break  # Only one image per class

In [29]:
from model import ImageEncoder
from ultis import predict_image_v2
import torch

model = ImageEncoder()
model.load_state_dict(torch.load(
    "/kaggle/input/image-encoder-leafclip/image_encoder_leaf_clip.pth", map_location="cpu"))
model.eval()


ImageEncoder(
  (swin): FeatureListNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (layers_0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate='none')

In [30]:
results = []
for class_name, img_path in rows:
    preds = predict_image_v2(model, img_path, "class_description.json", top_k=2)
    # preds is a list like [('Apple scab', 0.82), ('Healthy leaf', 0.10)]
    top1, top2 = preds[0][0], preds[1][0]
    results.append((img_path, class_name, top1, top2))


In [31]:
import csv

with open('predictions.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_path', 'true_class', 'pred1', 'score1(%)', 'pred2', 'score2(%)'])

    for class_name, img_path in rows:
        preds = predict_image_v2(model, img_path, "class_description.json", top_k=2)
        # preds is a list like [('Apple leaf', 0.8234), ('Healthy leaf', 0.0921)]
        (text1, prob1), (text2, prob2) = preds
        score1 = f"{prob1*100:.2f}"
        score2 = f"{prob2*100:.2f}"
        writer.writerow([img_path, class_name, text1, score1, text2, score2])

print(f"Saved results for {len(rows)} images to predictions.csv")


Saved results for 28 images to predictions.csv


In [32]:
import pandas as pd

df = pd.read_csv('predictions.csv')
# Show the first 5 rows
df.head()

# Show all rows (if small)
display(df)  # or just 'df' in the last cell

,image_path,true_class,pred1,score1(%),pred2,score2(%)
0,/kaggle/input/plantdoc-dataset/train/Tomato_le...,Tomato_leaf_yellow_virus,An image of Tomato leaves diseased by virosis ...,97.13,An image of Cowpea leaves diseased by mosaic v...,1.94
1,/kaggle/input/plantdoc-dataset/train/Tomato_le...,Tomato_leaf_late_blight,An image of Potato leaves diseased by late bli...,85.49,An image of Tomato leaves diseased by early bl...,14.17
2,/kaggle/input/plantdoc-dataset/train/Apple_Sca...,Apple_Scab_Leaf,An image of Apple leaves diseased by rust with...,88.42,An image of Apple leaves diseased by cedar app...,11.58
3,/kaggle/input/plantdoc-dataset/train/Cherry_le...,Cherry_leaf,An image of healthy Apple leaves showing of Le...,98.40,An image of healthy Cashew leaves showing of L...,1.60
4,/kaggle/input/plantdoc-dataset/train/grape_lea...,grape_leaf,An image of healthy Grape leaves showing of Le...,100.00,An image of healthy Strawberry leaves showing ...,0.00
5,/kaggle/input/plantdoc-dataset/train/Raspberry...,Raspberry_leaf,An image of healthy Strawberry leaves showing ...,100.00,An image of healthy Raspberry leaves showing o...,0.00
6,/kaggle/input/plantdoc-dataset/train/Soyabean_...,Soyabean_leaf,An image of healthy Soybean leaves showing of ...,99.96,An image of healthy Chili leaves showing of Le...,0.04
7,/kaggle/input/plantdoc-dataset/train/Corn_Gray...,Corn_Gray_leaf_spot,An image of Maize leaves diseased by cercospor...,100.00,An image of Maize leaves diseased by northern ...,0.00
8,/kaggle/input/plantdoc-dataset/train/Tomato_Ea...,Tomato_Early_blight_leaf,An image of Tomato leaves diseased by leaf mol...,97.44,An image of Tomato leaves diseased by virosis ...,1.60
9,/kaggle/input/plantdoc-dataset/train/Corn_leaf...,Corn_leaf_blight,An image of Maize leaves diseased by northern ...,99.98,An image of Maize leaves diseased by helmintho...,0.02
